In [1]:
import json
import os
import re
import numpy as np
from sentence_transformers import SentenceTransformer
import faiss

d:\chalkid\Uni\Academic\Sem 4\Artificial Intelligence\UAS\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
# Add this diagnostic cell before the main processing
def check_directory_structure(base_dir):
    """Check what files and folders exist in the base directory"""
    print(f"Checking directory structure for: {base_dir}")
    
    if not os.path.exists(base_dir):
        print(f"Base directory '{base_dir}' does not exist!")
        return
    
    for root, dirs, files in os.walk(base_dir):
        level = root.replace(base_dir, '').count(os.sep)
        indent = ' ' * 2 * level
        print(f"{indent}{os.path.basename(root)}/")
        subindent = ' ' * 2 * (level + 1)
        for file in files:
            print(f"{subindent}{file}")

# Check current structure
check_directory_structure(BASE_DIR)

Checking directory structure for: dataset/SistemOperasi
SistemOperasi/
  outline_operating_systems.txt
  Pertemuan_01_Foundations_Intro/
    materi_pertemuan01.txt
  Pertemuan_02_Structure_Services_Interaction/
    materi_pertemuan02.txt
  Pertemuan_03_Process_Management/
    materi_pertemuan03.txt
  Pertemuan_04_CPU_Scheduling/
    materi_pertemuan04.txt


In [12]:
# --- Konfigurasi ---
BASE_DIR = r"dataset/SistemOperasi"  # Direktori utama mata kuliah Anda
OUTLINE_FILE = os.path.join(BASE_DIR, "outline_operating_systems.txt")
OUTPUT_JSON_CHUNKS = os.path.join(BASE_DIR, "processed_chunks_with_metadata.json")
OUTPUT_FAISS_INDEX = os.path.join(BASE_DIR, "vector_store.index")

EMBEDDING_MODEL_NAME = 'all-MiniLM-L6-v2'  # Menghasilkan vektor 384 dimensi

# Parameter untuk chunking
MAX_CHUNK_SIZE_CHARS = 1000  # Ukuran maksimal chunk sebelum dipecah lebih lanjut
CHUNK_OVERLAP_CHARS = 150   # Jumlah karakter tumpang tindih antar sub-chunk
# Pola regex untuk mendeteksi heading (misal: # Judul, ## Sub Judul, dst.)
HEADING_SPLIT_PATTERN = r"(^\#{1,6}\s+.*$)" # Tangkap baris yang dimulai dengan 1-6 '#' diikuti spasi dan teks

# --- Fungsi Helper ---

def parse_outline(outline_filepath):
    """
    Mem-parsing file outline untuk mendapatkan informasi setiap pertemuan.
    Mengasumsikan format KEY: VALUE dan pemisah antar pertemuan adalah baris kosong.
    """
    pertemuan_list = []
    try:
        with open(outline_filepath, 'r', encoding='utf-8') as f:
            content = f.read()

        # Pisahkan berdasarkan blok pertemuan (diasumsikan dipisah oleh 'PERTEMUAN:')
        # dan pastikan ada MATAKULIAH di awal
        if not content.strip().startswith("MATAKULIAH:"):
            print(f"Peringatan: Format file outline '{outline_filepath}' mungkin tidak sesuai (tidak ada 'MATAKULIAH:').")
            # return [] # Bisa dihentikan jika format ketat

        # Menggunakan regex untuk menangkap blok pertemuan dengan lebih fleksibel
        # Pola ini mencari "PERTEMUAN:" dan mengambil semua baris hingga "PERTEMUAN:" berikutnya atau akhir file
        pertemuan_blocks = re.split(r'\nPERTEMUAN:', '\n' + content.split('PERTEMUAN:', 1)[-1] if 'PERTEMUAN:' in content else '')

        for block in pertemuan_blocks:
            if not block.strip():
                continue

            current_pertemuan = {}
            lines = block.strip().splitlines()

            # Ambil ID Pertemuan dari baris pertama blok (setelah "PERTEMUAN:")
            if lines:
                pertemuan_id_match = re.match(r'^\s*(\d+)', lines[0])
                if pertemuan_id_match:
                    current_pertemuan['id'] = int(pertemuan_id_match.group(1))
                else:
                    print(f"Peringatan: Tidak bisa parse ID Pertemuan dari blok: {lines[0]}")
                    continue # Lewati blok ini jika ID tidak bisa diparse

                # Proses sisa baris untuk KEY: VALUE
                for line in lines: # Mulai dari baris pertama lagi untuk key lain juga
                    if ":" in line:
                        key, value = line.split(":", 1)
                        key = key.strip().lower().replace(" ", "_")
                        value = value.strip()
                        if key == "file_materi" and not value: # Jika FILE_MATERI kosong
                            current_pertemuan[key] = None
                        else:
                            current_pertemuan[key] = value

            if 'id' in current_pertemuan and 'judul' in current_pertemuan and 'file_materi' in current_pertemuan :
                 # Pastikan file materi tidak None atau string kosong untuk dimasukkan
                if current_pertemuan.get('file_materi'):
                    pertemuan_list.append(current_pertemuan)
                elif current_pertemuan.get('file_materi') is None or not current_pertemuan.get('file_materi','').strip() :
                    print(f"Info: Pertemuan {current_pertemuan.get('id','N/A')} ({current_pertemuan.get('judul','Tanpa Judul')}) tidak memiliki file materi, akan dilewati untuk RAG.")

    except FileNotFoundError:
        print(f"Error: File outline '{outline_filepath}' tidak ditemukan.")
    except Exception as e:
        print(f"Error saat mem-parsing file outline: {e}")

    print(f"Berhasil mem-parsing {len(pertemuan_list)} pertemuan dengan file materi dari outline.")
    return pertemuan_list


def read_material_text(material_filepath):
    """Membaca konten teks dari file."""
    try:
        with open(material_filepath, 'r', encoding='utf-8') as f:
            return f.read()
    except FileNotFoundError:
        print(f"Error: File materi '{material_filepath}' tidak ditemukan.")
        return ""
    except Exception as e:
        print(f"Error saat membaca file materi '{material_filepath}': {e}")
        return ""

def _split_text_block_sliding_window(text_block, pertemuan_id, pertemuan_judul, heading, max_size, overlap):
    """
    Helper untuk memecah blok teks yang panjang menggunakan sliding window karakter.
    """
    final_chunks = []
    text_block_stripped = text_block.strip()
    if not text_block_stripped:
        return []

    if len(text_block_stripped) <= max_size:
        final_chunks.append({
            "pertemuan_id": pertemuan_id,
            "pertemuan_judul": pertemuan_judul,
            "original_heading": heading,
            "chunk_text": text_block_stripped
        })
    else:
        start_index = 0
        doc_len = len(text_block_stripped)
        while start_index < doc_len:
            end_index = start_index + max_size
            current_slice = text_block_stripped[start_index:min(end_index, doc_len)]

            chunk_text_final = current_slice.strip()

            if chunk_text_final:
                final_chunks.append({
                    "pertemuan_id": pertemuan_id,
                    "pertemuan_judul": pertemuan_judul,
                    "original_heading": heading,
                    "chunk_text": chunk_text_final
                })

            if min(end_index, doc_len) >= doc_len:
                break

            start_index += (max_size - overlap)
            start_index = max(0, start_index)
            if start_index >= doc_len:
                break

    return final_chunks

def chunk_material_heading_aware(text_content, pertemuan_id, pertemuan_judul):
    """
    Memecah konten materi menjadi chunks, mempertimbangkan heading sebagai pemisah alami.
    Jika teks di bawah satu heading terlalu panjang, akan dipecah lebih lanjut.
    """
    processed_chunks = []
    if not text_content or not text_content.strip():
        return []

    # Pisahkan teks berdasarkan heading, sambil mempertahankan headingnya.
    # re.split dengan capturing group (...) akan mempertahankan delimiter.
    parts = re.split(HEADING_SPLIT_PATTERN, text_content, flags=re.MULTILINE)

    current_heading = "Umum" # Default untuk konten sebelum heading pertama
    accumulated_text_for_section = ""

    for i, part in enumerate(parts):
        part_stripped = part.strip()
        if not part_stripped:
            continue

        # Cek apakah part ini adalah heading (berdasarkan pola regex)
        is_current_part_a_heading = re.match(HEADING_SPLIT_PATTERN, part_stripped, flags=re.MULTILINE)

        if is_current_part_a_heading:
            # Jika ada teks yang sudah terakumulasi untuk section SEBELUMNYA, proses dulu
            if accumulated_text_for_section.strip():
                sub_chunks = _split_text_block_sliding_window(accumulated_text_for_section,
                                                              pertemuan_id, pertemuan_judul, current_heading,
                                                              MAX_CHUNK_SIZE_CHARS, CHUNK_OVERLAP_CHARS)
                processed_chunks.extend(sub_chunks)

            current_heading = part_stripped  # Update heading saat ini
            accumulated_text_for_section = "" # Reset akumulator teks
        else:
            # Ini adalah konten di bawah heading saat ini
            accumulated_text_for_section += part_stripped + "\n" # Tambahkan newline agar antar paragraf tidak menyatu

    # Proses sisa teks yang terakumulasi untuk section terakhir
    if accumulated_text_for_section.strip():
        sub_chunks = _split_text_block_sliding_window(accumulated_text_for_section,
                                                      pertemuan_id, pertemuan_judul, current_heading,
                                                      MAX_CHUNK_SIZE_CHARS, CHUNK_OVERLAP_CHARS)
        processed_chunks.extend(sub_chunks)

    return processed_chunks

def get_text_embeddings(list_of_chunk_texts, model_name=EMBEDDING_MODEL_NAME):
    """Mengubah daftar teks chunk menjadi vektor embeddings."""
    if not list_of_chunk_texts:
        print("Tidak ada teks untuk di-embed.")
        return np.array([])
    try:
        print(f"Memuat model embedding: {model_name}...")
        embedding_model = SentenceTransformer(model_name)
        print("Model embedding berhasil dimuat.")
        print(f"Memulai proses embedding untuk {len(list_of_chunk_texts)} chunk teks...")
        embeddings = embedding_model.encode(list_of_chunk_texts, show_progress_bar=True)
        print(f"Proses embedding selesai. Dihasilkan {embeddings.shape[0]} embeddings dengan dimensi {embeddings.shape[1]}.")
        return embeddings
    except Exception as e:
        print(f"Error saat membuat embeddings: {e}")
        return np.array([])

def create_and_save_faiss_index(embeddings_np_array, index_output_path=OUTPUT_FAISS_INDEX):
    """Membuat FAISS index dan menyimpannya."""
    if embeddings_np_array.size == 0 or embeddings_np_array.ndim != 2:
        print("Array embedding kosong atau formatnya salah. FAISS index tidak dibuat.")
        return
    dimension = embeddings_np_array.shape[1]
    try:
        print(f"Membuat FAISS index dengan dimensi {dimension}...")
        index = faiss.IndexFlatL2(dimension)
        index.add(embeddings_np_array.astype('float32'))
        faiss.write_index(index, index_output_path)
        print(f"FAISS index dengan {index.ntotal} vektor berhasil dibuat dan disimpan ke: {index_output_path}")
    except Exception as e:
        print(f"Error saat membuat atau menyimpan FAISS index: {e}")

# --- Proses Utama Skrip ---
if __name__ == "__main__":
    print("Memulai Prosesor RAG: Persiapan Data...")

    # 1. Parse file outline mata kuliah
    print(f"\nLangkah 1: Mem-parsing outline dari '{OUTLINE_FILE}'...")
    daftar_pertemuan = parse_outline(OUTLINE_FILE)

    if not daftar_pertemuan:
        print("Tidak ada informasi pertemuan yang valid dari outline. Proses dihentikan.")
    else:
        all_processed_chunks_with_metadata = []

        # 2. Loop setiap pertemuan untuk membaca materi dan melakukan chunking
        print(f"\nLangkah 2: Memproses materi per pertemuan...")
        for pertemuan_info in daftar_pertemuan:
            pertemuan_id = pertemuan_info.get('id')
            judul_pertemuan = pertemuan_info.get('judul', f"Pertemuan {pertemuan_id}")
            file_materi_rel_path = pertemuan_info.get('file_materi')

            if not file_materi_rel_path:
                print(f"Info: Pertemuan ID {pertemuan_id} ({judul_pertemuan}) tidak memiliki path file materi. Dilewati.")
                continue

            file_materi_abs_path = os.path.join(BASE_DIR, file_materi_rel_path)
            print(f"  Memproses: Pertemuan {pertemuan_id} - '{judul_pertemuan}' dari file '{file_materi_abs_path}'")

            materi_text = read_material_text(file_materi_abs_path)
            if materi_text:
                chunks_for_this_pertemuan = chunk_material_heading_aware(materi_text, pertemuan_id, judul_pertemuan)
                all_processed_chunks_with_metadata.extend(chunks_for_this_pertemuan)
                print(f"    Dihasilkan {len(chunks_for_this_pertemuan)} chunk untuk pertemuan ini.")
            else:
                print(f"    Tidak ada konten teks yang dibaca dari '{file_materi_abs_path}'.")

        print(f"\nTotal chunk yang diproses dari semua pertemuan: {len(all_processed_chunks_with_metadata)}")

        if all_processed_chunks_with_metadata:
            # Ekstrak hanya teks chunk untuk proses embedding
            list_of_chunk_texts_for_embedding = [chunk['chunk_text'] for chunk in all_processed_chunks_with_metadata]

            # 3. Buat Embeddings
            print(f"\nLangkah 3: Membuat embeddings untuk semua chunk teks...")
            document_embeddings = get_text_embeddings(list_of_chunk_texts_for_embedding)

            if document_embeddings.size > 0:
                # 4. Buat dan Simpan FAISS Index
                print(f"\nLangkah 4: Membuat dan menyimpan FAISS index...")
                create_and_save_faiss_index(document_embeddings, OUTPUT_FAISS_INDEX)

                # 5. Simpan semua chunks beserta metadatanya ke file JSON
                print(f"\nLangkah 5: Menyimpan semua chunk yang diproses (dengan metadata) ke JSON...")
                try:
                    with open(OUTPUT_JSON_CHUNKS, "w", encoding="utf-8") as f:
                        json.dump(all_processed_chunks_with_metadata, f, ensure_ascii=False, indent=2)
                    print(f"Semua chunk yang diproses berhasil disimpan ke: {OUTPUT_JSON_CHUNKS}")
                except Exception as e:
                    print(f"Error saat menyimpan chunks ke JSON: {e}")
            else:
                print("Pembuatan FAISS index dan penyimpanan JSON chunks dibatalkan karena tidak ada embeddings yang valid.")
        else:
            print("Tidak ada chunk yang diproses sama sekali. Pastikan file materi ada dan berisi teks.")

    print("\n--- Prosesor RAG: Persiapan Data Selesai ---")

Memulai Prosesor RAG: Persiapan Data...

Langkah 1: Mem-parsing outline dari 'dataset/SistemOperasi\outline_operating_systems.txt'...
Berhasil mem-parsing 4 pertemuan dengan file materi dari outline.

Langkah 2: Memproses materi per pertemuan...
  Memproses: Pertemuan 1 - 'Foundations & Overview of Operating Systems' dari file 'dataset/SistemOperasi\Pertemuan_01_Foundations_Intro/materi_pertemuan_01.txt'
    Dihasilkan 13 chunk untuk pertemuan ini.
  Memproses: Pertemuan 2 - 'OS Components, Services, and Structure' dari file 'dataset/SistemOperasi\Pertemuan_02_Structure_Services_Interaction/materi_pertemuan_02.txt'
    Dihasilkan 12 chunk untuk pertemuan ini.
  Memproses: Pertemuan 3 - 'Process Management' dari file 'dataset/SistemOperasi\Pertemuan_03_Process_Management/materi_pertemuan_03.txt'
    Dihasilkan 14 chunk untuk pertemuan ini.
  Memproses: Pertemuan 4 - 'CPU Scheduling Algorithms' dari file 'dataset/SistemOperasi\Pertemuan_04_CPU_Scheduling/materi_pertemuan_04.txt'
    Diha

Batches: 100%|██████████| 2/2 [00:13<00:00,  6.51s/it]

Proses embedding selesai. Dihasilkan 56 embeddings dengan dimensi 384.

Langkah 4: Membuat dan menyimpan FAISS index...
Membuat FAISS index dengan dimensi 384...
FAISS index dengan 56 vektor berhasil dibuat dan disimpan ke: dataset/SistemOperasi\vector_store.index

Langkah 5: Menyimpan semua chunk yang diproses (dengan metadata) ke JSON...
Semua chunk yang diproses berhasil disimpan ke: dataset/SistemOperasi\processed_chunks_with_metadata.json

--- Prosesor RAG: Persiapan Data Selesai ---
